In [4]:
# Imports + load minimalist EDA file
import pandas as pd
from pathlib import Path
import numpy as np
path = Path("data/analysis/servicelines_house_with_attributes.csv")


In [5]:
import pandas as pd

# --- Normalization helpers ---
def normalize_status_material(val):
    if pd.isna(val):
        return "Unknown"
    s = str(val).strip().lower()

    # Lead or galvanized requiring replacement → Lead/GRR
    if "lead" == s or "galvanized requiring replacement" in s:
        return "Lead/GRR"

    # Any mention of unknown
    if "unknown" in s:
        return "Unknown"

    # Clear non-lead materials
    if (
        "non-lead" in s
        or "copper" in s
        or "plastic" in s
        or "other" in s
        or s == "galvanized"
    ):
        return "Non-Lead"

    # Default fallback
    return "Unknown"


def normalize_status_both(val):
    if pd.isna(val):
        return "Unknown"
    s = str(val).strip().lower()

    # Check for non-lead first to avoid catching "non-lead" as "lead"
    if "non-lead" in s:
        return "Non-Lead"

    # Lead or galvanized requiring replacement → Lead/GRR
    if "lead" in s or "galvanized requiring replacement" in s:
        return "Lead/GRR"

    return "Unknown"


# --- Main code ---
df = pd.read_csv(path)

# Convert timestamps if present
if "CreationDate" in df.columns:
    df["CreationDate_dt"] = pd.to_datetime(df["CreationDate"], unit="ms", errors="coerce")
if "EditDate" in df.columns:
    df["EditDate_dt"] = pd.to_datetime(df["EditDate"], unit="ms", errors="coerce")

# Normalize materials and statuses
if "custmaterial" in df.columns:
    df["custmaterial_cat"] = df["custmaterial"].apply(normalize_status_material)
if "utilmaterial" in df.columns:
    df["utilmaterial_cat"] = df["utilmaterial"].apply(normalize_status_material)
if "bothsidesstatus" in df.columns:
    df["bothsidesstatus_cat"] = df["bothsidesstatus"].apply(normalize_status_both)

# Print summaries
for col in ["custmaterial_cat", "utilmaterial_cat", "bothsidesstatus_cat"]:
    if col in df.columns:
        print(f"\nValue counts for {col}:")
        counts = df[col].value_counts(dropna=False)
        perc = (counts / len(df) * 100).round(1)
        print(pd.DataFrame({"count": counts, "percent": perc}))


C:\Users\bradk\AppData\Local\Temp\ipykernel_16000\690644486.py:48: DtypeWarning: Columns (31,32,33,34,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)



Value counts for custmaterial_cat:
                  count  percent
custmaterial_cat                
Unknown           65768     80.4
Non-Lead          15503     19.0
Lead/GRR            502      0.6

Value counts for utilmaterial_cat:
                  count  percent
utilmaterial_cat                
Non-Lead          48599     59.4
Unknown           31389     38.4
Lead/GRR           1785      2.2

Value counts for bothsidesstatus_cat:
                     count  percent
bothsidesstatus_cat                
Unknown              65742     80.4
Non-Lead             13770     16.8
Lead/GRR              2261      2.8


In [6]:
# When was this file last updated? 
print("Data last edited on:", df["EditDate_dt"].max())
# Show the ten most recent edit dates
print("Ten most recent edit dates:")
print(df["EditDate_dt"].nlargest(10))

Data last edited on: 2025-10-21 02:30:50.253000
Ten most recent edit dates:
55858   2025-10-21 02:30:50.253
56158   2025-10-21 02:30:50.253
56340   2025-10-21 02:30:50.253
56361   2025-10-21 02:30:50.253
56592   2025-10-21 02:30:50.253
56969   2025-10-21 02:30:50.253
57010   2025-10-21 02:30:50.253
57965   2025-10-21 02:30:50.253
58221   2025-10-21 02:30:50.253
58248   2025-10-21 02:30:50.253
Name: EditDate_dt, dtype: datetime64[ns]


In [7]:
# Print value counts for pipe material variables

for col in ["custmaterial", "custmaterial_cat", "utilmaterial", "utilmaterial_cat", "bothsidesstatus","bothsidesstatus_cat"]:
    print(f"\nValue counts for {col}:")
    counts = df[col].value_counts(dropna=False)
    percents = df[col].value_counts(normalize=True, dropna=False) * 100
    result = pd.DataFrame({"count": counts, "percent": percents.round(1)})
    print(result)



Value counts for custmaterial:
                                  count  percent
custmaterial                                    
Unknown - Material Unknown        65766     80.4
Non-Lead - Other                  13553     16.6
Non-Lead - Copper                  1055      1.3
Non-Lead - Plastic                  625      0.8
Lead                                499      0.6
Non-Lead                            181      0.2
Galvanized                           89      0.1
Galvanized Requiring Replacement      3      0.0
Unknown                               2      0.0

Value counts for custmaterial_cat:
                  count  percent
custmaterial_cat                
Unknown           65768     80.4
Non-Lead          15503     19.0
Lead/GRR            502      0.6

Value counts for utilmaterial:
                            count  percent
utilmaterial                              
Non-Lead - Copper           36591     44.7
Unknown - Material Unknown  31354     38.3
Non-Lead - Other        

In [8]:
# Print buckets for lead score by model classification for customer pipes

bins = (df.groupby("ModelClassification_Cust")["ModelScoreForLeadClassification_Cust"]
          .agg(["min","max","mean"]))
print(bins)


                               min       max      mean
ModelClassification_Cust                              
0.0                       0.004344  0.909811  0.087411
1.0                       0.004710  0.989992  0.305548
2.0                       0.011884  0.991866  0.559681
3.0                       0.042580  0.997360  0.752784


In [9]:
df.groupby("custmaterial_cat")[["ModelScoreForLeadClassification_Cust", "ModelClassification_Cust"]].agg(["min", "max", "mean"])


ModelScoreForLeadClassification_Cust                      \
                                                  min       max      mean   
custmaterial_cat                                                            
Lead/GRR                                     0.032780  0.995604  0.746242   
Non-Lead                                     0.005680  0.995908  0.371253   
Unknown                                      0.004344  0.997360  0.410813   

                 ModelClassification_Cust                 
                                      min  max      mean  
custmaterial_cat                                          
Lead/GRR                              0.0  3.0  2.474104  
Non-Lead                              0.0  3.0  1.062569  
Unknown                               0.0  3.0  1.487463

In [10]:
#Impute missing customer material categories based on model classification

# # Copy your clean category column
df["custmaterial_cat_imputed"] = df["custmaterial_cat"]

# Only impute for unknowns
mask = df["custmaterial_cat"] == "Unknown"

# Define mapping from model class to imputed category
class_map = {0: "Non-Lead", 1: "Non-Lead", 2: "Lead/GRR", 3: "Lead/GRR"}

df.loc[mask, "custmaterial_cat_imputed"] = (
    df.loc[mask, "ModelClassification_Cust"]
    .map(class_map)
    .fillna("Unknown")
)

#Impute missing utility material categories based on model classification

# # Copy your clean category column
df["utilmaterial_cat_imputed"] = df["utilmaterial_cat"]

# Only impute for unknowns
mask = df["utilmaterial_cat"] == "Unknown"

# Define mapping from model class to imputed category
class_map = {0: "Non-Lead", 1: "Non-Lead", 2: "Lead/GRR", 3: "Lead/GRR"}

df.loc[mask, "utilmaterial_cat_imputed"] = (
    df.loc[mask, "ModelClassification_City"]
    .map(class_map)
    .fillna("Unknown")
)

print()

def impute_bothsides(row):
    if "Lead/GRR" in (row["custmaterial_cat_imputed"], row["utilmaterial_cat_imputed"]):
        return "Lead/GRR"
    elif "Non-Lead" in (row["custmaterial_cat_imputed"], row["utilmaterial_cat_imputed"]):
        return "Non-Lead"
    else:
        return "Unknown"

df["bothsidesstatus_imputed"] = df.apply(impute_bothsides, axis=1)

In [11]:
# Comparing imputed vs original categories for lead pipes

for col in ["custmaterial_cat","custmaterial_cat_imputed", "utilmaterial_cat", "utilmaterial_cat_imputed","bothsidesstatus","bothsidesstatus_imputed"]:
    print(f"\nValue counts for {col}:")
    counts = df[col].value_counts(dropna=False)
    percents = df[col].value_counts(normalize=True, dropna=False) * 100
    result = pd.DataFrame({"count": counts, "percent": percents.round(1)})
    print(result)


Value counts for custmaterial_cat:
                  count  percent
custmaterial_cat                
Unknown           65768     80.4
Non-Lead          15503     19.0
Lead/GRR            502      0.6

Value counts for custmaterial_cat_imputed:
                          count  percent
custmaterial_cat_imputed                
Non-Lead                  52588     64.3
Lead/GRR                  29184     35.7
Unknown                       1      0.0

Value counts for utilmaterial_cat:
                  count  percent
utilmaterial_cat                
Non-Lead          48599     59.4
Unknown           31389     38.4
Lead/GRR           1785      2.2

Value counts for utilmaterial_cat_imputed:
                          count  percent
utilmaterial_cat_imputed                
Non-Lead                  67407     82.4
Lead/GRR                  14366     17.6

Value counts for bothsidesstatus:
                                  count  percent
bothsidesstatus                                 
Unknown 

In [12]:
#Using our three new imputed columns, custmaterial_cat_imputed, utilmaterial_cat_imputed, and bothsidesstatus_imputed   
# calculate the number and percent of lead pipes by neighborhood (neighborhood_name)


#custmaterial
lead_counts_custmaterial_imputed = df[df["custmaterial_cat_imputed"] == "Lead/GRR"].groupby("neighborhood_name").size()
total_counts = df.groupby("neighborhood_name").size()
lead_percents_custmaterial_imputed = (lead_counts_custmaterial_imputed / total_counts * 100).round(1)
non_lead_counts_custmaterial_imputed = df[df["custmaterial_cat_imputed"] == "Non-Lead"].groupby("neighborhood_name").size()
non_lead_percents_custmaterial_imputed = (non_lead_counts_custmaterial_imputed / total_counts * 100).round(1)

#utilmaterial
lead_counts_utilmaterial_imputed = df[df["utilmaterial_cat_imputed"] == "Lead/GRR"].groupby("neighborhood_name").size()
lead_percents_utilmaterial_imputed = (lead_counts_utilmaterial_imputed / total_counts * 100).round(1)
non_lead_counts_utilmaterial_imputed = df[df["utilmaterial_cat_imputed"] == "Non-Lead"].groupby("neighborhood_name").size()
non_lead_percents_utilmaterial_imputed = (non_lead_counts_utilmaterial_imputed / total_counts * 100).round(1)

#Bothsides
lead_counts_bothsides_imputed = df[df["bothsidesstatus_imputed"] == "Lead/GRR"].groupby("neighborhood_name").size()
lead_percents_bothsides_imputed = (lead_counts_bothsides_imputed / total_counts * 100).round(1)
non_lead_counts_bothsides_imputed = df[df["bothsidesstatus_imputed"] == "Non-Lead"].groupby("neighborhood_name").size()
non_lead_percents_bothsides_imputed = (non_lead_counts_bothsides_imputed / total_counts * 100).round(1)

lead_imputed_summary = pd.DataFrame({"total_count": total_counts, 
                                      "lead_count_bothsides_imputed": lead_counts_bothsides_imputed, "lead_percent_bothsides_imputed": lead_percents_bothsides_imputed,
                                      "lead_count_custmaterial_imputed": lead_counts_custmaterial_imputed, "lead_percent_custmaterial_imputed": lead_percents_custmaterial_imputed,
                                      "lead_count_utilmaterial_imputed": lead_counts_utilmaterial_imputed, "lead_percent_utilmaterial_imputed": lead_percents_utilmaterial_imputed,
                                      "non_lead_count_bothsides_imputed": non_lead_counts_bothsides_imputed, "non_lead_percent_bothsides_imputed": non_lead_percents_bothsides_imputed,
                                      "non_lead_count_custmaterial_imputed": non_lead_counts_custmaterial_imputed, "non_lead_percent_custmaterial_imputed": non_lead_percents_custmaterial_imputed,
                                      "non_lead_count_utilmaterial_imputed": non_lead_counts_utilmaterial_imputed, "non_lead_percent_utilmaterial_imputed": non_lead_percents_utilmaterial_imputed})
lead_imputed_summary = lead_imputed_summary.fillna(0)
print(lead_imputed_summary)


#Output to CSV
output_path = "data/analysis/neighborhoods_lead_imputed_summary.csv"
lead_imputed_summary.to_csv(output_path)


                        total_count  lead_count_bothsides_imputed  \
neighborhood_name                                                   
Ancarows Landing                205                          13.0   
Beaufont                        468                          56.0   
Belle and Mayo Islands           21                           0.0   
Bellemeade                     1058                         636.0   
Bellevue                       1334                         919.0   
...                             ...                           ...   
Windsor Farms                   619                          21.0   
Woodhaven                       560                         168.0   
Woodland Heights                852                         662.0   
Woodville                       206                         130.0   
Worthington                     481                          10.0   

                        lead_percent_bothsides_imputed  \
neighborhood_name                           